Test du projet de class trajets indirect

In [1]:
%load_ext autoreload
%autoreload 2
import sys #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
import traitement_od as t
import pandas as pd
import numpy as np
import altair as alt
import datetime as dt
pd.set_option('display.max_colwidth', 1000)
import os
from sklearn.cluster import DBSCAN

IMPORT DONNEES GLOBALES (SUPPRESSION DES DOUBLONS EN MM TEMPS)

In [353]:
%%time df_3semaines=t.ouvrir_fichier_lapi('2019-01-28 00:00:00','2019-02-17 23:59:59')
df_3semaines=df_3semaines.drop_duplicates(['created','immat']))
df_3semaines.set_index('created').sort_index()

Wall time: 2min 3s


IMPORT DES DONNEES SUR UNE JOURNEE

In [350]:
df_journee=t.ouvrir_fichier_lapi('2019-01-28','2019-01-30')

In [352]:
#redresser la valeur de l
df_journee.loc[df_journee['camera_id']==15,'l']=1

LISTE DES TRAJETS

In [3]:
liste_trajet_locale=(pd.DataFrame([{'origine':'A63','destination':'A10','cam_o':15, 'cam_d':11, 'trajets':[
                                                        {'cameras':[15,19,4,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[15,19,1,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[15,4,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[15,1,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[15,19,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[15,19,4,11],'type_trajet':'indirect'},
                                                        {'cameras':[15,19,1,11],'type_trajet':'indirect'},
                                                        {'cameras':[15,4,11],'type_trajet':'indirect'},
                                                        {'cameras':[15,1,11],'type_trajet':'indirect'},
                                                        {'cameras':[15,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[15,19,11],'type_trajet':'indirect'},
                                                        {'cameras':[15,11],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A10','destination':'A63','cam_o':12, 'cam_d':13,'trajets':[{'cameras':[12,6,2,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,6,3,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,6,2,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,6,3,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,6,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,2,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,3,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,2,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,3,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,6,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[12,13],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A63','destination':'N10','cam_o':15, 'cam_d':5,'trajets':[{'cameras':[15,19,4,5],'type_trajet':'indirect'},
                                                        {'cameras':[15,19,1,5],'type_trajet':'indirect'},
                                                        {'cameras':[15,4,5],'type_trajet':'indirect'},
                                                        {'cameras':[15,1,5],'type_trajet':'indirect'},
                                                        {'cameras':[15,19,5],'type_trajet':'indirect'}, 
                                                        {'cameras':[15,5],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'N10','destination':'A63','cam_o':6, 'cam_d':13,'trajets':[{'cameras':[6,2,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[6,3,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[6,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[6,2,13],'type_trajet':'indirect'},
                                                        {'cameras':[6,3,13],'type_trajet':'indirect'},
                                                        {'cameras':[6,13],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A62','destination':'A10','cam_o':10, 'cam_d':11,'trajets':[{'cameras':[10,4,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[10,4,11],'type_trajet':'indirect'},
                                                        {'cameras':[10,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[10,11],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A10','destination':'A62','cam_o':12, 'cam_d':9,'trajets':[{'cameras':[12,6,3,9],'type_trajet':'indirect'},
                                                        {'cameras':[12,3,9],'type_trajet':'indirect'},
                                                        {'cameras':[12,6,9],'type_trajet':'indirect'},
                                                        {'cameras':[12,9],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A62','destination':'N10','cam_o':10, 'cam_d':5,'trajets':[{'cameras':[10,4,5],'type_trajet':'indirect'},
                                                        {'cameras':[10,5],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'N10','destination':'A62','cam_o':6, 'cam_d':9,'trajets':[{'cameras':[6,3,9],'type_trajet':'indirect'},
                                                        {'cameras':[6,9],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A63','destination':'A62','cam_o':15, 'cam_d':9,'trajets':[{'cameras':[15,19,9],'type_trajet':'indirect'},
                                                        {'cameras':[15,9],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A62','destination':'A63','cam_o':10, 'cam_d':13,'trajets':[{'cameras':[10,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[10,13],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A89','destination':'A63','cam_o':8 ,'cam_d':13,'trajets':[{'cameras':[8,3,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[8,18,13],'type_trajet':'indirect'},
                                                        {'cameras':[8,3,13],'type_trajet':'indirect'},
                                                        {'cameras':[8,13],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A63','destination':'A89','cam_o':15, 'cam_d':7,'trajets':[{'cameras':[15,19,4,7],'type_trajet':'indirect'},
                                                        {'cameras':[15,4,7],'type_trajet':'indirect'},
                                                        {'cameras':[15,19,7],'type_trajet':'indirect'},
                                                        {'cameras':[15,7],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A89','destination':'A62','cam_o':8, 'cam_d':9,'trajets':[{'cameras':[8,3,9],'type_trajet':'indirect'},
                                                        {'cameras':[8,9],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A62','destination':'A89','cam_o':10, 'cam_d':7,'trajets':[{'cameras':[10,4,7],'type_trajet':'indirect'},
                                                        {'cameras':[10,7],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A89','destination':'A10','cam_o':8, 'cam_d':11,'trajets':[{'cameras':[8,5,11],'type_trajet':'indirect'},
                                                        {'cameras':[8,11],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A10','destination':'A89','cam_o':12, 'cam_d':7,'trajets':[{'cameras':[12,6,7],'type_trajet':'indirect'},
                                                        {'cameras':[12,7],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A89','destination':'N10','cam_o':8, 'cam_d':5,'trajets':[{'cameras':[8,5],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'N10','destination':'A89','cam_o':6, 'cam_d':7,'trajets':[{'cameras':[6,7],'type_trajet':'direct'}
                                                       ]},
                            {'origine':'A10','destination':'A660','cam_o':12, 'cam_d':18,'trajets':[{'cameras':[12,6,2,18],'type_trajet':'indirect'},
                                                         {'cameras':[12,6,3,18],'type_trajet':'indirect'},
                                                         {'cameras':[12,2,18],'type_trajet':'indirect'},
                                                         {'cameras':[12,3,18],'type_trajet':'indirect'},
                                                         {'cameras':[12,6,18],'type_trajet':'indirect'},
                                                         {'cameras':[12,18],'type_trajet':'direct'},
                                                        ]},
                            {'origine':'A660','destination':'A10','cam_o':19, 'cam_d':18,'trajets':[{'cameras':[19,4,5,11],'type_trajet':'indirect'},
                                                         {'cameras':[19,1,5,11],'type_trajet':'indirect'},
                                                         {'cameras':[19,1,11],'type_trajet':'indirect'},
                                                         {'cameras':[19,4,11],'type_trajet':'indirect'},
                                                         {'cameras':[19,5,11],'type_trajet':'indirect'},
                                                         {'cameras':[19,11],'type_trajet':'direct'},
                                                       ]},
                            {'origine':'A660','destination':'A62','cam_o':19, 'cam_d':9,'trajets':[{'cameras':[19,9],'type_trajet':'direct'}
                                                        ]},
                            {'origine':'A62','destination':'A660','cam_o':10, 'cam_d':18,'trajets':[{'cameras':[10,18],'type_trajet':'direct'}
                                                        ]},
                            {'origine':'A660','destination':'A89','cam_o':19, 'cam_d':7,'trajets':[{'cameras':[19,4,7],'type_trajet':'indirect'},
                                                         {'cameras':[19,7],'type_trajet':'direct'}
                                                        ]},
                            {'origine':'A89','destination':'A660','cam_o':8, 'cam_d':18,'trajets':[{'cameras':[8,3,18],'type_trajet':'indirect'},
                                                         {'cameras':[8,18],'type_trajet':'direct'}
                                                        ]},
                            {'origine':'N10','destination':'A660','cam_o':6, 'cam_d':18,'trajets':[{'cameras':[6,2,18],'type_trajet':'indirect'},
                                                         {'cameras':[6,3,18],'type_trajet':'indirect'},
                                                         {'cameras':[6,18],'type_trajet':'direct'},
                                                        ]},
                            {'origine':'A660','destination':'N10','cam_o':19, 'cam_d':5,'trajets':[{'cameras':[19,1,5],'type_trajet':'indirect'},
                                                         {'cameras':[19,4,5],'type_trajet':'indirect'},
                                                         {'cameras':[19,5],'type_trajet':'direct'},
                                                        ]}
                           ]))[['origine', 'destination', 'cam_o', 'cam_d','trajets']]

In [655]:
liste_complete_trajet=pd.DataFrame([[tuple(liste_cam['cameras']),value[0],value[1],value[2],value[3],len(liste_cam['cameras'])] for obj, value in liste_trajet_locale.iterrows() for liste_cam in value[4]],
             columns=['cameras','origine','destination','cam_o','cam_d','nb_cams'])
liste_complete_trajet=liste_complete_trajet.sort_values('nb_cams',ascending=False)
liste_complete_trajet


(15, 11)
(15, 11)
(12, 13)
(12, 13)
(12, 13)
(6, 13)
(15, 7)
(10, 11)
(15, 5)
(15, 5)
(8, 13)
(12, 9)
(19, 18)
(12, 13)
(12, 13)
(12, 13)
(12, 13)
(19, 18)
(12, 18)
(12, 18)
(15, 11)
(15, 11)
(15, 11)
(15, 11)
(15, 11)
(6, 13)
(15, 7)
(10, 13)
(19, 7)
(15, 9)
(8, 13)
(8, 13)
(19, 18)
(15, 7)
(8, 9)
(10, 7)
(19, 18)
(8, 11)
(8, 18)
(12, 7)
(19, 18)
(12, 18)
(12, 18)
(6, 9)
(10, 11)
(10, 5)
(15, 5)
(15, 11)
(15, 11)
(15, 11)
(15, 11)
(19, 5)
(12, 13)
(12, 13)
(12, 13)
(12, 13)
(19, 5)
(15, 5)
(15, 5)
(6, 13)
(6, 13)
(6, 13)
(6, 18)
(10, 11)
(6, 18)
(12, 9)
(12, 9)
(12, 18)
(19, 9)
(8, 18)
(19, 18)
(6, 18)
(10, 18)
(12, 18)
(19, 7)
(6, 9)
(6, 7)
(8, 5)
(15, 11)
(12, 13)
(15, 5)
(6, 13)
(10, 11)
(12, 9)
(10, 5)
(15, 9)
(10, 13)
(8, 13)
(15, 7)
(8, 9)
(10, 7)
(8, 11)
(12, 7)
(19, 5)


In [730]:
donnees_trajet=t.trajet(df_3semaines.loc[pd.to_datetime('2019-01-28 16:00:00'):pd.to_datetime('2019-01-28 16:00:00')+pd.Timedelta(hours=18)],'2019-01-28 16:00:00',60,(10,18), type='Global')

In [733]:
donnees_trajet.df_transit.loc[donnees_trajet.df_transit['immat']=='2da9c14f66a652882b65149a257de9f25f5a5a24']

,date_cam_1,immat,state,l,date_cam_2,cameras,origine,destination,o_d,tps_parcours
9,2019-01-28 16:04:43,2da9c14f66a652882b65149a257de9f25f5a5a24,BE,1,2019-01-28 16:18:56,"(10, 18)",A62,A660,A62-A660,00:14:13


CALCUL DES O-D SUR UNE JOURNEE, AVEC RENVOI DES DF O-D OU PASSAGES 

In [353]:
dico_trajet_od, dico_passag_od, dico_od,  dico_passag=t.transit_1_jour(df_journee,'2019-01-28 08:00:00', liste_trajet_locale)

date : 2019-01-28 08:00:00 debut_traitement : 2019-04-05 15:32:42.058239
date : 2019-01-28 09:00:00 debut_traitement : 2019-04-05 15:35:25.209336


IMPORTANT : VERIF DES DOUBLONS ENTRE LES TRAJETS

In [803]:
df_depart=dico_od.loc[dico_od['destination']=='N10'].copy()
jointure=df_depart.merge(dico_od, on='immat')
jointure=jointure.loc[jointure.date_cam_1_x!=jointure.date_cam_1_y].copy()
(jointure.loc[((jointure.date_cam_1_y>=jointure.date_cam_1_x) & (jointure.date_cam_1_y<=jointure.date_cam_2_x)) |
              ((jointure.date_cam_2_y>=jointure.date_cam_1_x) & (jointure.date_cam_2_y<=jointure.date_cam_2_x))])

,date_cam_1_x,immat,state_x,l_x,date_cam_2_x,cameras_x,origine_x,destination_x,o_d_x,tps_parcours_x,date_cam_1_y,state_y,l_y,date_cam_2_y,cameras_y,origine_y,destination_y,o_d_y,tps_parcours_y
1385,2019-01-28 10:13:59,0c48c7cb179c4a15c18f2d4bb5434a7d2611e463,BE/SM,1,2019-01-28 10:17:38,"(10, 5)",A62,N10,A62-N10,00:03:39,2019-01-28 10:33:53,BE/SM,1,2019-01-28 10:17:38,"(10, 5)",A62,N10,A62-N10,-1 days +23:43:45


CREER LE TABLEAU DE MATRICE O-D

In [796]:
#MATRICE O-D !!II!
pd.pivot_table(dico_od,values='l', index='origine', columns='destination',aggfunc='count', margins=True )

destination,A10,A62,A63,A660,A89,N10,All
origine,,,,,,,
A10,NaN,392.0,262.0,189.0,48.0,NaN,891
A62,179.0,NaN,55.0,107.0,56.0,217.0,614
A63,540.0,31.0,NaN,NaN,155.0,2555.0,3281
A660,179.0,79.0,NaN,NaN,117.0,423.0,798
A89,49.0,136.0,199.0,176.0,NaN,130.0,690
N10,NaN,294.0,841.0,315.0,63.0,NaN,1513
All,947.0,932.0,1357.0,787.0,439.0,3325.0,7787


POURCENTAGE PL PAR PLAGE HORAIRE ET CAMERA

In [432]:
#isoler pl source
groupe_tv_type_simple=df_journee.groupby('immat')['l'].apply(lambda x:test_unicite_type(list(x),mode='1/2')).reset_index().drop_duplicates('immat')
df_pl=df_journee.join(groupe_tv_type_simple.loc[groupe_tv_type_simple['l']==1].set_index('immat'),on='immat',how='inner',rsuffix='_right')
df_pl.set_index('created',inplace=True)

#obtenir les nb de pl par heure et par camera sur la source
df_synthese_pl_tot=df_pl.groupby('camera_id').resample('H').count()['immat'].rename(column={'immat':'nb_pl_tot'})
df_synthese_pl_transit=dico_passag.set_index('created').groupby('camera_id').resample('H').count()['immat'].rename(column={'immat':'nb_pl_transit'})
df_pct_pl_transit=pd.concat([df_synthese_pl_tot,df_synthese_pl_transit], axis=1, join='inner')
df_pct_pl_transit.columns=[['nb_pl_tot','nb_pl_transit']]
df_pct_pl_transit['pct_pl_transit']=df_pct_pl_transit.apply(lambda x : float(x['nb_pl_transit'])*100 / x['nb_pl_tot'] ,axis=1)
df_pct_pl_transit

TESTS DONNEES COMPLETES

In [795]:
dico_od,  dico_passag=t.transit_temps_complet_v2('2019-01-28 00:00:00',21,liste_trajet_locale,df_3semaines)

date : 2019-01-28 00:00:00 debut_traitement : 2019-04-10 17:03:16.123663
date : 2019-01-28 01:00:00 debut_traitement : 2019-04-10 17:03:17.107814
date : 2019-01-28 02:00:00 debut_traitement : 2019-04-10 17:03:17.748291
date : 2019-01-28 03:00:00 debut_traitement : 2019-04-10 17:03:18.435637
date : 2019-01-28 04:00:00 debut_traitement : 2019-04-10 17:03:19.122981
date : 2019-01-28 05:00:00 debut_traitement : 2019-04-10 17:03:20.044679
date : 2019-01-28 06:00:00 debut_traitement : 2019-04-10 17:03:21.606797
date : 2019-01-28 07:00:00 debut_traitement : 2019-04-10 17:03:23.372020
date : 2019-01-28 08:00:00 debut_traitement : 2019-04-10 17:03:25.902704
date : 2019-01-28 09:00:00 debut_traitement : 2019-04-10 17:03:28.870775
date : 2019-01-28 10:00:00 debut_traitement : 2019-04-10 17:03:31.760750
date : 2019-01-28 11:00:00 debut_traitement : 2019-04-10 17:03:34.635097
date : 2019-01-28 12:00:00 debut_traitement : 2019-04-10 17:03:37.400096
date : 2019-01-28 13:00:00 debut_traitement : 2019-

In [791]:
#verifs que les destination A660
dico_od.loc[(dico_od['destination']=='A660') & (dico_od['state']!='FR')].head(25)

,date_cam_1,immat,state,l,date_cam_2,cameras,origine,destination,o_d,tps_parcours
17,2019-01-28 16:08:40,121854636dbabbed787ff8c970c881b76914736c,NL,1,2019-01-28 16:54:40,"(12, 6, 3, 18)",A10,A660,A10-A660,00:46:00
24,2019-01-28 16:13:31,3c2d530b2ae44b912d53a4aef63f4b90a95dbbb9,UK,1,2019-01-28 17:14:04,"(12, 6, 3, 18)",A10,A660,A10-A660,01:00:33
49,2019-01-28 16:43:42,bc1cc90b5317eebb3c43b94579421897fbcfe0f7,BE,1,2019-01-28 17:29:51,"(12, 6, 3, 18)",A10,A660,A10-A660,00:46:09
63,2019-01-28 16:54:24,e5f454f10badd472ef30f3db5a4612829e8a9701,FR/IT,1,2019-01-28 17:30:56,"(12, 6, 18)",A10,A660,A10-A660,00:36:32
10,2019-01-28 16:06:33,78541775498c1c799942d3fa9c6d6ba8df9025d4,PL,1,2019-01-29 08:03:10,"(8, 18)",A89,A660,A89-A660,15:56:37
20,2019-01-28 16:17:21,374d1fdcf6a24dbeff9dd9aef98e23a625a3dded,PT,1,2019-01-28 16:55:27,"(8, 18)",A89,A660,A89-A660,00:38:06
9,2019-01-28 16:04:43,2da9c14f66a652882b65149a257de9f25f5a5a24,BE,1,2019-01-28 16:18:56,"(10, 18)",A62,A660,A62-A660,00:14:13
17,2019-01-28 16:15:40,ce104bac054266180202343084f1e623f1ea897d,PT,1,2019-01-29 08:09:39,"(10, 18)",A62,A660,A62-A660,15:53:59
24,2019-01-28 16:25:55,0dc09c0c3445550835f8d788f40e0ad4476c6ce1,ES,1,2019-01-28 16:35:15,"(10, 18)",A62,A660,A62-A660,00:09:20
30,2019-01-28 16:43:17,b833fee2a13a3216739bdaf496f7b877dd072a3a,BG,1,2019-01-29 06:22:17,"(10, 18)",A62,A660,A62-A660,13:39:00


In [804]:
# #retrouver une immatriculatiob dans les données completes
df_3semaines.loc[df_3semaines['immat']=='0c48c7cb179c4a15c18f2d4bb5434a7d2611e463']

,camera_id,immat,fiability,l,state
created,,,,,
2019-01-28 09:19:07,5,0c48c7cb179c4a15c18f2d4bb5434a7d2611e463,56,0,BE/SM
2019-01-28 09:33:34,5,0c48c7cb179c4a15c18f2d4bb5434a7d2611e463,78,1,BE/SM
2019-01-28 10:13:59,10,0c48c7cb179c4a15c18f2d4bb5434a7d2611e463,66,1,BE/SM
2019-01-28 10:17:38,5,0c48c7cb179c4a15c18f2d4bb5434a7d2611e463,47,1,BE/SM
2019-01-28 10:33:53,10,0c48c7cb179c4a15c18f2d4bb5434a7d2611e463,77,1,BE/SM
2019-01-28 12:19:08,5,0c48c7cb179c4a15c18f2d4bb5434a7d2611e463,78,1,BE/SM
2019-01-28 13:11:48,5,0c48c7cb179c4a15c18f2d4bb5434a7d2611e463,77,1,BE/SM
2019-01-28 13:31:56,5,0c48c7cb179c4a15c18f2d4bb5434a7d2611e463,46,0,BE/SM
2019-01-28 15:46:24,4,0c48c7cb179c4a15c18f2d4bb5434a7d2611e463,45,-1,BE


In [762]:
#verif sur un trajet a un horaire donnee sur un trajet global
dico_od.loc[dico_od['immat']=='2da9c14f66a652882b65149a257de9f25f5a5a24']

,date_cam_1,immat,state,l,date_cam_2,cameras,origine,destination,o_d,tps_parcours
9,2019-01-28 16:04:43,2da9c14f66a652882b65149a257de9f25f5a5a24,BE,1,2019-01-28 16:18:56,"(10, 18)",A62,A660,A62-A660,00:14:13
102,2019-01-28 16:30:13,2da9c14f66a652882b65149a257de9f25f5a5a24,BE,1,2019-01-28 16:18:56,"(10, 18)",A62,A660,A62-A660,-1 days +23:48:43


In [570]:
test.df_transit.o_d.unique()

array(['N10-A660', 'N10-A62', 'N10-A63', 'N10-A89', 'A10-A62', 'A10-A660',
       'A10-A63', 'A10-A89'], dtype=object)

In [ ]:
#verif a un horaire donne sur un trajet simple
%time test=t.trajet(df_3semaines.loc['2019-01-29 21:00:00':'2019-01-30 15:00:00'], '2019-01-29 21:00:00', 60, 16, [19,4])

In [135]:
#pour test des fonctions de trajets directs
df_duree=df_3semaines.loc['2019-01-29 21:00:00':'2019-01-30 15:00:00'].loc['2019-01-29 21:00:00':'2019-01-29 22:00:00']
df_duree_cam1=df_duree.loc[df_duree.loc[:,'camera_id']==19]
df_duree_autres_cam=df_3semaines.loc['2019-01-29 21:00:00':'2019-01-30 15:00:00'].loc[(df_3semaines.loc['2019-01-29 21:00:00':'2019-01-30 15:00:00'].loc[:,'immat'].isin(df_duree_cam1.loc[:,'immat']))]
cam1_croise_autre_cam=df_duree_cam1.reset_index().merge(df_duree_autres_cam.reset_index(), on='immat')
cam1_croise_suivant=cam1_croise_autre_cam.loc[(cam1_croise_autre_cam.loc[:,'created_x']<cam1_croise_autre_cam.loc[:,'created_y'])]
cam1_fastest_next=cam1_croise_suivant.loc[cam1_croise_suivant.groupby(['immat'])['created_y'].idxmin()]

In [ ]:
#pour test des fonctions de trajets global 
liste_trajet_od=[(15,19,4,5),(15,5),(15,4,5),(15,19,5)]
camera1, camera2=15, 5
df=df_3semaines.loc['2019-01-28 00:00:00':'2019-01-29 23:59:59']
df_duree=df.loc['2019-01-28 00:00:00':'2019-01-28 01:00:00']
df_duree_cam1=df_duree.loc[df_duree.loc[:,'camera_id']==camera1]
df_duree_autres_cam=df.loc[(df.loc[:,'immat'].isin(df_duree_cam1.loc[:,'immat']))]
df_autres_cam_temp_ok=df_duree_autres_cam.loc['2019-01-29 08:00:00':'2019-01-30 08:00:00']
groupe=(df_autres_cam_temp_ok.sort_index().reset_index().groupby('immat').agg({'camera_id':lambda x : tuple(x), 'l': lambda x : test_unicite_type(list(x),'1/2'),
                                                                        'created':lambda x: tuple(x)}))

groupe['camera_id']=groupe.apply(lambda x : filtrer_passage(x['camera_id'],df_liste_trajet),axis=1)

groupe['created']=groupe.apply(lambda x : recuperer_date_cam2(x['camera_id'],x['created'],liste_trajet_od),axis=1)

df_agrege=df_duree_cam1.join(groupe,on='immat',lsuffix='_left')[['immat', 'l','camera_id','created']].rename(columns={'created':'date_cam_2', 'camera_id':'cameras'})
df_agrege=df_agrege.reset_index().rename(columns={'created':'date_cam_1'})
df_agrege['tps_parcours']=df_agrege.apply(lambda x : x.date_cam_2-x.date_cam_1, axis=1)

df_trajet=df_agrege.loc[df_agrege['cameras'].isin(liste_trajet_od)]
df_trajet_final=(df_trajet.loc[(df_trajet['cameras'].apply(lambda x : x[-1])==camera2) & (df_trajet['l']==1)
                                  & (df_trajet['tps_parcours'] < pd.Timedelta('11:30:00'))])

In [810]:
%%time
camera1, camera2=10,18
df=df_3semaines.loc['2019-01-28 10:00:00':'2019-01-29 23:59:59']
df_duree=df.loc['2019-01-28 10:00:00':'2019-01-28 11:00:00']
df_duree_cam1=df_duree.loc[df_duree.loc[:,'camera_id']==camera1]
df_duree_autres_cam=df.loc[(df.loc[:,'immat'].isin(df_duree_cam1.loc[:,'immat']))]
groupe=(df_duree_autres_cam.sort_index().reset_index().groupby('immat').agg({'camera_id':lambda x : tuple(x), 'l': lambda x : test_unicite_type(list(x),'1/2'),
                                                                               'created':lambda x: tuple(x)}))
groupe_pl=groupe.loc[groupe['l']==1].copy()
groupe_pl['camera_id']=groupe_pl.apply(lambda x : filtrer_passage(x['camera_id'],liste_complete_trajet,camera1),axis=1)#on filtre les cameras selon la liste des trajets existants
groupe_pl['created']=groupe_pl.apply(lambda x : recuperer_date_cam2(x['camera_id'],x['created'],liste_complete_trajet,camera1),axis=1)
df_ts_trajets=(groupe_pl.reset_index().merge(liste_complete_trajet[['cameras','origine','destination']],right_on='cameras', left_on='camera_id').
               rename(columns={'created':'date_cam_2'}).drop('camera_id',axis=1))
df_ts_trajets['o_d']=df_ts_trajets.apply(lambda x : x['origine']+'-'+x['destination'],axis=1)
df_agrege=df_duree_cam1.reset_index().merge(df_ts_trajets,on='immat').drop(['camera_id', 'l_x','state','fiability'],axis=1).rename(columns={'l_y':'l','created':'date_cam_1'})
df_agrege['tps_parcours']=df_agrege.apply(lambda x : x.date_cam_2-x.date_cam_1, axis=1)
df_agrege=df_agrege.loc[df_agrege['date_cam_2'] > df_agrege['date_cam_1']]

df_joint_passag_transit=df_agrege.merge(df_duree_autres_cam.reset_index(), on='immat')
df_passag_transit1=df_joint_passag_transit.loc[(df_joint_passag_transit.apply(lambda x : x['camera_id'] in x['cameras'], axis=1))]
df_passag_transit=(df_passag_transit1.loc[df_passag_transit1.apply(lambda x : x['date_cam_1']<=x['created']<=x['date_cam_2'], axis=1)]
                [['created','camera_id','immat','fiability','l_y','state']].rename(columns={'l_y':'l'}))

Wall time: 344 ms


In [811]:
#groupe_pl.loc[groupe_pl['created']>'2019-01-28 16:18:00'].sort_values('created')
#groupe.loc['2da9c14f66a652882b65149a257de9f25f5a5a24']
df_duree_cam1.loc[df_duree_cam1['immat']=='0c48c7cb179c4a15c18f2d4bb5434a7d2611e463']

,camera_id,immat,fiability,l,state
created,,,,,
2019-01-28 10:13:59,10,0c48c7cb179c4a15c18f2d4bb5434a7d2611e463,66,1,BE/SM
2019-01-28 10:33:53,10,0c48c7cb179c4a15c18f2d4bb5434a7d2611e463,77,1,BE/SM


In [812]:
df_agrege.loc[df_agrege['immat']=='0c48c7cb179c4a15c18f2d4bb5434a7d2611e463']

,date_cam_1,immat,l,date_cam_2,cameras,origine,destination,o_d,tps_parcours
14,2019-01-28 10:13:59,0c48c7cb179c4a15c18f2d4bb5434a7d2611e463,1,2019-01-28 10:17:38,"(10, 5)",A62,N10,A62-N10,00:03:39


In [785]:
def filtrer_passage(liste, df_liste_trajet, cam) :
    for liste_cams in [a for a in liste_complete_trajet.cameras.tolist() if a[0]==cam] :
        if liste[:len(liste_cams)]==tuple(liste_cams):
            return liste[:len(liste_cams)]
    else : return liste

In [786]:
def recuperer_date_cam2(liste,liste_created,df_liste_trajet, cam):
    for liste_cams in [a for a in liste_complete_trajet.cameras.tolist() if a[0]==cam] :
        if liste[:len(liste_cams)]==tuple(liste_cams):
            return liste_created[len(liste_cams)-1]
    else : return liste_created[-1]

In [784]:
[a for a in liste_complete_trajet.cameras.tolist() if a[0]==19]

[(19, 1, 5, 11),
 (19, 4, 5, 11),
 (19, 4, 7),
 (19, 4, 11),
 (19, 5, 11),
 (19, 1, 11),
 (19, 4, 5),
 (19, 1, 5),
 (19, 9),
 (19, 11),
 (19, 7),
 (19, 5)]

In [623]:
filtrer_passage((10, 4, 5, 3, 9, 4, 10, 5, 7, 8, 6, 3, 9),liste_complete_trajet)

(10, 4, 5)

In [760]:
for cameras in zip([15,12,8,10,19,6],range(6)):
    print (cameras)

(15, 0)
(12, 1)
(8, 2)
(10, 3)
(19, 4)
(6, 5)
